# Notebook 2: Data Cleaning

## Objective
Transform the raw startup funding dataset into a clean, analysis-ready format by:
- Parsing dates and extracting temporal features
- Cleaning and standardizing funding amounts
- Extracting funding stages from investment types
- Normalizing city names
- Counting investors from comma-separated lists
- Handling missing values appropriately

## Expected Output
Clean dataset saved as `../data/startup_funding_clean.csv` with additional derived columns ready for EDA.

In [44]:
# Import required libraries
import pandas as pd
import numpy as np
import warnings
import sys
import os

warnings.filterwarnings('ignore')

In [45]:
# Add scripts directory to path for importing helper modules
sys.path.append(os.path.abspath('../scripts'))

from stage_mapper import apply_stage_mapping
from amount_parser import process_amount_column

---
## Stage 1: Load Raw Data

In [46]:
# Load raw dataset
df = pd.read_csv('../data/raw/startup_funding.csv')
print(f"Original dataset shape: {df.shape}")
df.head(3)

Original dataset shape: (3044, 10)


,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN


---
## Stage 2: Date Parsing & Temporal Features

Parse the `Date dd/mm/yyyy` column and extract Year, Month, Quarter for temporal analysis.

In [47]:
# Parse dates (dayfirst=True for dd/mm/yyyy format)
df['Date'] = pd.to_datetime(df['Date dd/mm/yyyy'], dayfirst=True, errors='coerce')

# Extract temporal features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter

# Check results
print(f"Date parsing complete. Invalid dates: {df['Date'].isna().sum()}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
df[['Date dd/mm/yyyy', 'Date', 'Year', 'Month', 'Quarter']].head()

Date parsing complete. Invalid dates: 8
Date range: 2015-01-02 00:00:00 to 2020-01-13 00:00:00


,Date dd/mm/yyyy,Date,Year,Month,Quarter
0,09/01/2020,2020-01-09,2020.0,1.0,1.0
1,13/01/2020,2020-01-13,2020.0,1.0,1.0
2,09/01/2020,2020-01-09,2020.0,1.0,1.0
3,02/01/2020,2020-01-02,2020.0,1.0,1.0
4,02/01/2020,2020-01-02,2020.0,1.0,1.0


---
## Stage 3: Amount Cleaning & Standardization

The `Amount in USD` column is mislabeled—it contains **INR amounts** in Indian comma format. We'll:
1. Parse amounts using `amount_parser.py` helper
2. Create standardized columns: `Amount_INR`, `Amount_Lakhs`, `Amount_Crores`
3. Create log-transformed amount for modeling

In [48]:
# Apply amount parsing (creates Amount_INR, Amount_Lakhs, Amount_Crores, Funding_Amount_Log)
df = process_amount_column(df, amount_column='Amount in USD')

print(f"Amount parsing complete. Missing amounts: {df['Amount_INR'].isna().sum()}")
print(f"Amount range (INR): {df['Amount_INR'].min():,.0f} to {df['Amount_INR'].max():,.0f}")
df[['Amount in USD', 'Amount_INR', 'Amount_Lakhs', 'Amount_Crores']].head()

Amount parsing complete. Missing amounts: 967
Amount range (INR): 2 to 3,900,000,000


,Amount in USD,Amount_INR,Amount_Lakhs,Amount_Crores
0,"20,00,00,000",200000000.0,2000.00000,20.000000
1,"80,48,394",8048394.0,80.48394,0.804839
2,"1,83,58,860",18358860.0,183.58860,1.835886
3,"30,00,000",3000000.0,30.00000,0.300000
4,"18,00,000",1800000.0,18.00000,0.180000


---
## Stage 4: Funding Stage Extraction

Extract standardized funding stages from `InvestmentnType` column using `stage_mapper.py`:
- Maps variations (e.g., "Seed Round", "Seed Funding") to canonical "Seed"
- Creates `Stage_Order` (1-11) for ordinal modeling
- Handles "unknown" cases as "Undisclosed" (Stage_Order = 0)

In [49]:
# Apply stage mapping (creates Stage and Stage_Order columns)
df = apply_stage_mapping(df, inv_type_column='InvestmentnType')

print(f"Stage extraction complete. Undisclosed stages: {(df['Stage'] == 'Undisclosed').sum()}")
print("\nStage distribution:")
print(df['Stage'].value_counts().head(10))

Stage extraction complete. Undisclosed stages: 14

Stage distribution:
Stage
Seed              1539
Private Equity    1368
Debt Funding        30
Series A            24
Series B            21
Series D+           20
Series C            14
Undisclosed         14
Pre-Series A         9
Angel                3
Name: count, dtype: int64


In [50]:
# Verify stage ordering
df[['InvestmentnType', 'Stage', 'Stage_Order']].drop_duplicates().sort_values('Stage_Order').head(15)

,InvestmentnType,Stage,Stage_Order
20,Venture,Undisclosed,0
30,NaN,Undisclosed,0
35,Maiden Round,Undisclosed,0
90,Venture - Series Unknown,Undisclosed,0
47,Single Venture,Undisclosed,0
55,Venture Round,Undisclosed,0
31,Funding Round,Undisclosed,0
105,Inhouse Funding,Undisclosed,0
3000,Crowd funding,Undisclosed,0
3002,Crowd Funding,Undisclosed,0


### 🔧 **Fix Applied:** Stage Mapper Updated

The previous run showed `Stage_Order = -1` for "Other" category. This has been fixed in `stage_mapper.py`:
- **Private Equity variations** (Private, PrivateEquity, Private\nEquity) → Private Equity (9)
- **Equity/Mezzanine** → Private Equity (9)
- **Bridge Round** → Series D+ (8)
- **Term Loan** → Debt Funding (11)
- **Generic venture/funding rounds** → Undisclosed (0)
- **Series D-Z** → Series D+ (8)

**Action Required:** Re-run cells 3, 11, and 12 to apply the fix.

---
## Stage 5: City Normalization

Standardize city names to handle variations:
- "Bangalore" → "Bengaluru"
- "Bombay" → "Mumbai"
- Case inconsistencies (e.g., "DELHI" → "Delhi")

In [51]:
# City name standardization mapping
city_mapping = {
    'bangalore': 'Bengaluru',
    'bengaluru': 'Bengaluru',
    'bombay': 'Mumbai',
    'mumbai': 'Mumbai',
    'new delhi': 'Delhi',
    'delhi': 'Delhi',
    'ncr': 'Delhi',
    'gurgaon': 'Gurugram',
    'gurugram': 'Gurugram',
    'noida': 'Noida',
    'pune': 'Pune',
    'hyderabad': 'Hyderabad',
    'chennai': 'Chennai',
    'kolkata': 'Kolkata',
    'ahmedabad': 'Ahmedabad',
    'jaipur': 'Jaipur'
}

# Normalize city names
df['City_Clean'] = df['City  Location'].str.strip().str.lower()
df['City_Clean'] = df['City_Clean'].map(city_mapping).fillna(df['City_Clean'])
df['City_Clean'] = df['City_Clean'].str.title()  # Capitalize first letter

print(f"Unique cities before cleaning: {df['City  Location'].nunique()}")
print(f"Unique cities after cleaning: {df['City_Clean'].nunique()}")
print("\nTop 10 cities:")
print(df['City_Clean'].value_counts().head(10))

Unique cities before cleaning: 112
Unique cities after cleaning: 109

Top 10 cities:
City_Clean
Bengaluru    841
Mumbai       567
Delhi        455
Gurugram     337
Pune         105
Hyderabad     99
Chennai       97
Noida         92
Ahmedabad     38
Jaipur        30
Name: count, dtype: int64


---
## Stage 6: Investor Counting

Count the number of investors from the comma-separated `Investors Name` column.

In [52]:
# Count investors (split by comma)
df['Investor_Count'] = df['Investors Name'].fillna('').str.split(',').str.len()
df.loc[df['Investors Name'].isna(), 'Investor_Count'] = 0  # Set to 0 if missing

print(f"Investor count statistics:")
print(df['Investor_Count'].describe())
print(f"\nRecords with 0 investors: {(df['Investor_Count'] == 0).sum()}")
df[['Investors Name', 'Investor_Count']].head(10)

Investor count statistics:
count    3044.000000
mean        1.905716
std         1.413742
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        10.000000
Name: Investor_Count, dtype: float64

Records with 0 investors: 24


,Investors Name,Investor_Count
0,Tiger Global Management,1
1,Susquehanna Growth Equity,1
2,Sequoia Capital India,1
3,Vinod Khatumal,1
4,Sprout Venture Partners,1
5,Chiratae Ventures,1
6,Ant Financial,1
7,Sathguru Catalyzer Advisors,1
8,Ping An Global Voyager Fund,1
9,"Mumbai Angels, Ravikanth Reddy",2


---
## Stage 7: Handle Missing Values

Review and handle missing data strategically based on column importance.

In [53]:
# Check missing values in key columns
missing_summary = pd.DataFrame({
    'Column': ['Startup Name', 'Date', 'Amount_INR', 'Stage', 'City_Clean', 'Industry Vertical'],
    'Missing_Count': [
        df['Startup Name'].isna().sum(),
        df['Date'].isna().sum(),
        df['Amount_INR'].isna().sum(),
        df['Stage'].isna().sum(),
        df['City_Clean'].isna().sum(),
        df['Industry Vertical'].isna().sum()
    ]
})
missing_summary['Missing_Percentage'] = (missing_summary['Missing_Count'] / len(df) * 100).round(2)
print("Missing value summary:")
print(missing_summary)

Missing value summary:
              Column  Missing_Count  Missing_Percentage
0       Startup Name              0                0.00
1               Date              8                0.26
2         Amount_INR            967               31.77
3              Stage              0                0.00
4         City_Clean            180                5.91
5  Industry Vertical            171                5.62


In [54]:
# Drop rows where critical fields are missing
print(f"Shape before dropping missing values: {df.shape}")

# Drop rows with missing Startup Name (can't identify the record)
df = df.dropna(subset=['Startup Name'])

# Keep rows with missing amounts/stages for now (will handle in modeling phase)
print(f"Shape after dropping missing Startup Name: {df.shape}")
print(f"Records dropped: {3044 - len(df)}")

Shape before dropping missing values: (3044, 22)
Shape after dropping missing Startup Name: (3044, 22)
Records dropped: 0


---
## Stage 8: Final Data Quality Check

Verify the cleaned dataset and review final column structure.

In [55]:
# Final dataset info
print(f"Final cleaned dataset shape: {df.shape}")
print(f"\nColumn summary:")
df.info()

Final cleaned dataset shape: (3044, 22)

Column summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Sr No               3044 non-null   int64         
 1   Date dd/mm/yyyy     3044 non-null   object        
 2   Startup Name        3044 non-null   object        
 3   Industry Vertical   2873 non-null   object        
 4   SubVertical         2108 non-null   object        
 5   City  Location      2864 non-null   object        
 6   Investors Name      3020 non-null   object        
 7   InvestmentnType     3040 non-null   object        
 8   Amount in USD       2084 non-null   object        
 9   Remarks             419 non-null    object        
 10  Date                3036 non-null   datetime64[ns]
 11  Year                3036 non-null   float64       
 12  Month               3036 non-null   float64    

In [56]:
# Preview cleaned data
print("\nSample of cleaned data:")
df[['Startup Name', 'Date', 'Year', 'City_Clean', 'Stage', 'Stage_Order', 
    'Amount_Crores', 'Investor_Count']].head(10)


Sample of cleaned data:


,Startup Name,Date,Year,City_Clean,Stage,Stage_Order,Amount_Crores,Investor_Count
0,BYJU’S,2020-01-09,2020.0,Bengaluru,Private Equity,9,20.000000,1
1,Shuttl,2020-01-13,2020.0,Gurugram,Series C,7,0.804839,1
2,Mamaearth,2020-01-09,2020.0,Bengaluru,Series B,6,1.835886,1
3,https://www.wealthbucket.in/,2020-01-02,2020.0,Delhi,Pre-Series A,4,0.300000,1
4,Fashor,2020-01-02,2020.0,Mumbai,Seed,2,0.180000,1
5,Pando,2020-01-13,2020.0,Chennai,Series A,5,0.900000,1
6,Zomato,2020-01-10,2020.0,Gurugram,Private Equity,9,15.000000,1
7,Ecozen,2019-12-12,2019.0,Pune,Series A,5,0.600000,1
8,CarDekho,2019-12-06,2019.0,Gurugram,Series D+,8,7.000000,1
9,Dhruva Space,2019-12-03,2019.0,Bengaluru,Seed,2,5.000000,2


---
## Stage 9: Export Cleaned Data

Save the cleaned dataset for use in subsequent notebooks (EDA, Feature Engineering, Modeling).

In [57]:
# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

# Export cleaned data
output_path = '../data/startup_funding_clean.csv'
df.to_csv(output_path, index=False)
print(f" Cleaned data exported to: {output_path}")
print(f"   Shape: {df.shape}")

 Cleaned data exported to: ../data/startup_funding_clean.csv
   Shape: (3044, 22)


---
##  Summary of Cleaning Operations

### Transformations Applied:
1.  **Date Parsing:** Converted `dd/mm/yyyy` strings to datetime objects
2.  **Temporal Features:** Extracted Year, Month, Quarter
3.  **Amount Cleaning:** Parsed Indian format, created INR/Lakhs/Crores columns
4.  **Stage Extraction:** Mapped investment types to standardized stages (1-11 ordering)
5.  **City Normalization:** Standardized city names (Bangalore→Bengaluru, etc.)
6.  **Investor Counting:** Counted investors from comma-separated lists
7.  **Missing Value Handling:** Dropped rows with missing startup names

### New Columns Created:
- `Date` - Parsed datetime
- `Year`, `Month`, `Quarter` - Temporal features
- `Amount_INR`, `Amount_Lakhs`, `Amount_Crores` - Standardized amounts
- `Funding_Amount_Log` - Log-transformed amount (for modeling)
- `Stage` - Canonical funding stage name
- `Stage_Order` - Numerical stage ordering (0-11)
- `City_Clean` - Standardized city names
- `Investor_Count` - Number of investors

### Next Steps:
 **Notebook 3:** Exploratory Data Analysis (EDA)
- Temporal trends in funding
- City-wise distribution
- Industry analysis
- Stage-wise amount patterns
- Correlation analysis